In [5]:
import PyPDF2
import pandas as pd
from unidecode import unidecode
import re
import time
from datetime import datetime, timedelta

In [2]:
def extraeTextoPdf(pdfPath, textIni, textFin):
    with open(pdfPath, 'rb') as pdfFile:
        # Create a PDF reader object
        pdfReader = PyPDF2.PdfReader(pdfFile)
        
        num_pages = len(pdfReader.pages)
        text = ""
        for page_number in range(num_pages):
            page = pdfReader.pages[page_number]
            text += page.extract_text()

        variantsI = [textIni[:i] + " " + textIni[i:] for i in range(len(textIni))]
        variantsF = [textFin[:i] + " " + textFin[i:] for i in range(len(textFin))]
        for variant in variantsI:
            try:
                encuentra = text.index(variant)
                textIni = variant
            except:
                continue
    
        for variant in variantsF:
            try:
                encuentra = text.index(variant)
                textFin = variant
            except:
                continue

        # Iterate through each page in the PDF
        contPagesIni = 0
        for pageNum in range(len(pdfReader.pages)):
            # Extract text from the current page
            page = pdfReader.pages[pageNum]
            testText = page.extract_text()
            if textIni in testText:
                break
            else:
                contPagesIni += 1

        contPagesFin = 0
        for pageNum in range(len(pdfReader.pages)):
            # Extract text from the current page
            page = pdfReader.pages[pageNum]
            testText = page.extract_text()
            if textFin in testText:
                break
            else:
                contPagesFin += 1

            
        # Initialize a variable to store the extracted text
        extracted_text1 = ''
        for pageNum in range(contPagesIni,contPagesFin +1):
            # Extract text from the current page
            page2 = pdfReader.pages[pageNum]
            extracted_text1 += page2.extract_text()

        patron = re.compile(f'{textIni}(.*?){textFin}', re.DOTALL)
    
        resultado = patron.search(extracted_text1)
        extracted_text = resultado.group(1)
    
    return extracted_text

In [3]:
def convert_to_regex(input_strings):
    regex_patterns = []
    for string in input_strings:
        str1 = string[0] + "("
        for letter in string[1:-2]:
            let = '.*' + letter
            str1 = f'{str1}{let}'
        str2 = f'.*){string[-2]}(.*){string[-1]}'
        str1 = f'{str1}{str2}'
        regex_pattern = str1
        regex_patterns.append(regex_pattern)
    return regex_patterns

def convert_to_regex2(input_strings):
    regex_patterns = []
    for string in input_strings:
        str1 = "(.*)" + string[0] + "("
        for letter in string[1:-1]:
            let = '.*' + letter
            str1 = f'{str1}{let}'
        str2 = f'.*){string[-1]}(.*)'
        str1 = f'{str1}{str2}'
        regex_pattern = str1
        regex_patterns.append(regex_pattern)
    return regex_patterns


In [ ]:
# 35-49 | Desde: 1. Conflictos activos | TOTAL
# 50-51 | Desde: 1.1. Conflictos activos | Hasta: TOTAL
# 52-89 | Desde: 1.1 Conflictos activos | Hasta: TOTAL
# 90-125 | Desde: Detalle de los conflictos sociales en un departamento: | Hasta: Detalle de los conflictos sociales en más de un departamento:
# 126-134 | Desde: 4.1 CONFLICTOS | Hasta: 4.2 CONFLICTOS
# 135-140 | Desde: 5.1 CONFLICTOS | Hasta: 5.2 CONFLICTOS
# 141-171 | Desde: 5.1 CONFLICTOS | Hasta: 4.2 CONFLICTOS SOCIALES
# 172-235 | Desde: 5.1 CONFLICTOS | Hasta: 5.2 CONFLICTOS SOCIALES


In [3]:
dicTiposDocs = {'tipo1' : [35, 49, '1. Conflictos activos', 'TOTAL'],
                'tipo2' : [50, 51, '1.1. Conflictos activos', 'TOTAL'],
                'tipo3' : [52, 89, '1.1 Conflictos activos', 'TOTAL'],
                'tipo4' : [90, 125, 'Detalle de los conflictos sociales en un departamento:', 'Detalle de los conflictos sociales en más de un departamento:'],
                'tipo5' : [126, 134, '4.1 CONFLICTOS', '4.2 CONFLICTOS'],
                'tipo6' : [135, 140, '5.1 CONFLICTOS', '5.2 CONFLICTOS'],
                'tipo7' : [141, 171, '5.1 CONFLICTOS', '4.2 CONFLICTOS SOCIALES'],
                'tipo8' : [172, 235, '5.1 CONFLICTOS', '5.2 CONFLICTOS SOCIALES']
                }

#Tipo8 funciona desde el 178


In [4]:
def generate_month_year_dict(start_year, start_month, end_year, end_month):
    start_date = datetime(start_year, start_month, 1)
    end_date = datetime(end_year, end_month, 1)
    current_date = start_date
    result_list = []

    months_spanish = {
        1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio',
        7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'
    }

    while current_date <= end_date:
        year = current_date.year
        month = current_date.month
        monthYear = str(months_spanish[month]) + ' ' + str(year)
        result_list.append(monthYear)
        current_date = current_date + timedelta(days=32)
        current_date = current_date.replace(day=1)

    return result_list

In [161]:
def datosConflictos(nInforme):

    folder_path = "C:/Users/User/Documents/GitHub/CNHGitHub/Data Perú/Conflictos sociales/pdf_reports_conflictos_sociales/"
    #folder_path = "D:/1. Documentos/0. Bases de datos/0. Dofiles y Scripts/CNH_rep/Data Perú/Conflictos sociales/pdf_reports_conflictos_sociales/"

    pdfPath = folder_path + str(nInforme) + '.pdf'
    
    # En un solo departamento
    if nInforme > 171:
        textoConflicto = extraeTextoPdf(pdfPath = pdfPath, textIni = "5.1 CONFLICTOS", textFin= "5.2 CONFLICTOS")
    elif nInforme > 140 & nInforme <= 171:
        textoConflicto = extraeTextoPdf(pdfPath = pdfPath, textIni = "5.1 CONFLICTOS", textFin= "4.2 CONFLICTOS SOCIALES")
    elif nInforme > 134 & nInforme <= 140:
        textoConflicto = extraeTextoPdf(pdfPath = pdfPath, textIni = "5.1 CONFLICTOS", textFin= "5.2 CONFLICTOS")
    elif nInforme > 125 & nInforme <= 134:
        textoConflicto = extraeTextoPdf(pdfPath = pdfPath, textIni = "4.1 CONFLICTOS", textFin= "4.2 CONFLICTOS")
    elif nInforme > 89 & nInforme <= 125:
        textoConflicto = extraeTextoPdf(pdfPath = pdfPath, textIni = 'Detalle de los conflictos sociales en un departamento:', textFin= 'Detalle de los conflictos sociales en más de un departamento:')        
    else:
        print("No está programado todavía")

    result_dict = generate_month_year_dict(2007, 1, 2023, 12)

    textoConflicto = unidecode(textoConflicto)    
    
    departamentosPeru = [
        "Amazonas",
        "Ancash",
        "Apurimac",
        "Arequipa",
        "Ayacucho",
        "Cajamarca",
        "Callao",
        "Cusco",
        "Huancavelica",
        "Huanuco",
        "Ica",
        "Junin",
        "La Libertad",
        "Lambayeque",
        "Lima",
        "Loreto",
        "Madre de Dios",
        "Moquegua",
        "Pasco",
        "Piura",
        "Puno",
        "San Martin",
        "Tacna",
        "Tumbes",
        "Ucayali"
    ]
    
    results = {}
    
    # Identificar si los departamentos tienen conflictos
    dptos = []
    for dpto in departamentosPeru:
        try:
            encuentra = textoConflicto.index(dpto + "  ")
            dptos.append(dpto)
        except:
            continue
    
    del encuentra
    
    # Agrupación por dptos
    nDpto = 0
    for dpto in dptos:
        if dpto == dptos[-1]:
            inicio = textoConflicto.index("\n" + dptos[nDpto] + "  \n")
            fin = len(textoConflicto)
            results[dpto] = {textoConflicto[inicio + len(dptos[nDpto] + "  ")+1 :fin]}
        else:
            inicio = textoConflicto.index(dptos[nDpto] + "  ")
            fin = textoConflicto.index(dptos[nDpto + 1] + "  ")
            results[dpto] = {textoConflicto[inicio + len(dptos[nDpto] + "  ")+1 :fin]}
            nDpto = nDpto + 1
        
        if nInforme > 177:
            stopWords=["Tipo:","Ingreso como caso nuevo:", "Caso:","Ubicacion:","Actores primarios:", "Actores secundarios:","Actores terciarios:","NO HAY DIALOGO", " HAY DIALOGO", "Actores Primarios:", "Actores Secundarios:", "Actores Terciarios:", "Actor secundario:", "Actor primario:", "Actor terciario:"]
        else:
            stopWords=["Tipo:", "Caso:","Ubicacion:","Actores primarios:", "Actores secundarios:","Actores terciarios:","NO HAY DIALOGO", " HAY DIALOGO", "Actores Primarios:", "Actores Secundarios:", "Actores Terciarios:", "Actor secundario:", "Actor primario:", "Actor terciario:"]
            
        textoDpto = list(results[dpto])[0]
    
        textoDpto = textoDpto.replace("Descripcion  Hechos del mes", "")
        textoDpto = textoDpto.replace("Hechos del mes  Hechos del mes", "")
        textoDpto = textoDpto.replace("CASO NUEVO", "")
    
        for word in stopWords:
            variants = [word[:i] + " " + word[i:] for i in range(len(word))]
            for variant in variants:
                textoDpto = textoDpto.replace(variant,word)
    
        textoDpto = textoDpto.replace("Actores Primarios:", "Actores primarios:")
        textoDpto = textoDpto.replace("Actores Secundarios:", "Actores secundarios:")
        textoDpto = textoDpto.replace("Actor primario:", "Actores primarios:")
        textoDpto = textoDpto.replace("Actor secundario:", "Actores secundarios:")
        textoDpto = textoDpto.replace("Actores Terciarios:", "Actores terciarios:")
        textoDpto = textoDpto.replace("Actor terciario:", "Actores terciarios:")
        textoDpto = textoDpto.replace("NO HAY DIALOGO", "NOHAYDIALOGO")
        
        
        if nInforme > 177:
            stopWords=["Tipo:","Ingreso como caso nuevo:", "Caso:","Ubicacion:","Actores primarios:", "Actores secundarios:","Actores terciarios:","NOHAYDIALOGO", "HAY DIALOGO"]
        else:
            stopWords=["Tipo:", "Caso:","Ubicacion:","Actores primarios:", "Actores secundarios:","Actores terciarios:","NOHAYDIALOGO", "HAY DIALOGO"]
    
        texto = textoDpto
        texto = texto.split("Tipo:")
        texto = texto[1:]
        texto = ["Tipo:" + elem for elem in texto]

        regex_patterns = convert_to_regex(stopWords)
        for i, word in enumerate(stopWords):
            texto = [re.sub(regex_patterns[i], word, elem) for elem in texto]

        listEncabezados = [f"Reporte Mensual de Conflictos Sociales N.° {nInforme} - {result_dict[ nInforme - 35]}", f"Reporte Mensual de Conflictos Sociales N.° {nInforme}, {result_dict[ nInforme - 35]}", f"Reporte Mensual de Conflictos Sociales N.deg {nInforme}, {result_dict[ nInforme - 35]}", f"Reporte Mensual de Conflictos Sociales N.deg {nInforme} - {result_dict[ nInforme - 35]}", "ADJUNTIA PARA LA PREVENCION DE CONFLICTOS SOCIALES Y LA GOBERNABILIDAD - DEFENSORIA DEL PUEBLO"]

        regexEncabezados = convert_to_regex2(listEncabezados)
        for i, word in enumerate(listEncabezados):
            texto = [re.sub(regexEncabezados[i], "", elem) for elem in texto]

        cont = 0
        for tex in texto:
            texConf = texto[cont]
            for word in stopWords:
                if word == "NOHAYDIALOGO" :
                    texConf = texConf.replace(word, "|" + "Dialogo: NOHAYDIALOGO" + "|")
                elif word == "HAY DIALOGO":
                    texConf = texConf.replace(word, "|" + "Dialogo: HAYDIALOGO" + "|")
                else:
                    texConf = texConf.replace(word, "|" + word)
            texConf = texConf.split("|")
            texConf = texConf[1:]
            texto[cont] = texConf
            cont = cont +1
        
        results[dpto] = texto
        
    resultsIncomp = {}
    
    dptoList = []
    tipoList = []
    ingresoList = []
    casoList = []
    ubiList = []
    actPrimList = []
    actSecList = []
    actTerList = []
    dialogList = []
    detalleList = []
    
    for dpto in dptos:
        listConf = results[dpto]
        if nInforme > 177:
            nCol = 9
        else:
            nCol = 8
        results[dpto] = [sorted(conf) for conf in listConf if len(conf) == nCol]
        resultsIncomp[dpto] = [sorted(conf) for conf in listConf if len(conf) != nCol]
        listConf = results[dpto]
    
        for conf in listConf:
            dptoList.append(dpto)
            detalleList.append(conf[0])
            actPrimList.append(conf[1])
            actSecList.append(conf[2])
            actTerList.append(conf[3])
            casoList.append(conf[4])
            dialogList.append(conf[5])
            if nCol == 9:
                ingresoList.append(conf[6])
                tipoList.append(conf[7])
                ubiList.append(conf[8])
            else:
                tipoList.append(conf[6])
                ingresoList.append("")
                ubiList.append(conf[7])
    
    for dpto in resultsIncomp.keys():
        if len(resultsIncomp[dpto]) != 0:
            for conf in range(0, len(resultsIncomp[dpto])):
                texto = "|".join(resultsIncomp[dpto][conf])
                try:
                    encuentra = texto.index("Actores secundarios:")
                    encuentra = texto.index("Actores terciarios:")
                    texto = texto + f'|Ingreso como caso nuevo: {result_dict[nInforme - 36]}'
                    resultsIncomp[dpto][conf] = texto.split('|')
                except:
                    try:
                        encuentra = texto.index("Actores terciarios:")
                        texto = texto + "|Actores secundarios: -"
                        resultsIncomp[dpto][conf] = texto.split('|')
                    except:
                        texto = texto + "|Actores terciarios: -"
                        resultsIncomp[dpto][conf] = texto.split('|')                     
                        continue
                    continue
            
            listConf = resultsIncomp[dpto]
            resultsIncomp[dpto] = [sorted(conf) for conf in listConf if len(conf) == nCol]
            listConf = resultsIncomp[dpto]
            
            for conf in listConf:
                dptoList.append(dpto)
                detalleList.append(conf[0])
                actPrimList.append(conf[1])
                actSecList.append(conf[2])
                actTerList.append(conf[3])
                casoList.append(conf[4])
                dialogList.append(conf[5])
                if nCol == 9:
                    ingresoList.append(conf[6])
                    tipoList.append(conf[7])
                    ubiList.append(conf[8])
                else:
                    tipoList.append(conf[6])
                    ubiList.append(conf[7])
                    ingresoList.append("")
        else:
            continue
    
    
    tiposUnicos = ['Socioambiental',
                   'Asuntos de gobierno nacional',
                   'Asuntos de gobierno regional',
                   'Asuntos de gobierno local',
                   'Asunto de gobierno nacional',
                   'Asunto de gobierno regional',
                   'Asunto de gobierno local',
                   'Comunal',
                   'Otros asuntos',
                   'Laboral',
                   'Demarcacion territorial',
                   'Cultivo ilegal de hoja de coca',
                   'Asuntos de Gobiernos Regional',
                   'Asuntos de Gobierno Local',
                   'Asuntos de Gobiernos Nacional',
                   'Tipo otros asuntos',
                   'Demarcacion Territorial',
                   'Por asuntos de gobierno local',
                   'Por asuntos de gobierno nacional',
                   'Por asuntos de gobierno regional'
                   ]
    regexTipos = convert_to_regex2(tiposUnicos)
    for i, word in enumerate(tiposUnicos):
        for n, elem in enumerate(tipoList):
            tipoList[n] = re.sub(regexTipos[i], word, elem)

    listas = [dptoList, tipoList, ingresoList ,casoList, ubiList, actPrimList, actSecList, actTerList, dialogList, detalleList]
    keyList = ["Dpto", "Tipo", "Ingreso como caso nuevo", "Caso", "Ubicacion","Actores primarios", "Actores secundarios", "Actores terciarios","Dialogo", "Detalle del mes"]

    patrones = [":  ", ": "]

    for i, lista in enumerate(listas):
        for patron in patrones:
            col = keyList[i] + patron
            listas[i] = [tex.replace(col,"") for tex in lista]

    patrones = [".  \n", ". \n", " . \n", ".   \n", "  \n", ": "]

    for patron in patrones:
        for i, lista in enumerate(listas):
            listas[i] = [tex.replace(patron,"") for tex in lista]
            
    diccionario = {key: lista for key, lista in zip(keyList, listas)}
    data = pd.DataFrame(diccionario)
    data['Mes'] = result_dict[nInforme - 35]
    data['NInforme'] = nInforme
    return data, results, resultsIncomp, textoConflicto

In [118]:
start_time = time.time()
df = datosConflictos(178)
for n in range(179,236):
    df = pd.concat([datosConflictos(n),df])
    print(f"Listo n° {n}")

end_time = time.time()
tiempo = end_time - start_time
print(f"Tiempo transcurrido: {tiempo}")

Listo n° 179
Listo n° 180
Listo n° 181
Listo n° 182
Listo n° 183
Listo n° 184
Listo n° 185
Listo n° 186
Listo n° 187
Listo n° 188
Listo n° 189
Listo n° 190
Listo n° 191
Listo n° 192
Listo n° 193
Listo n° 194
Listo n° 195
Listo n° 196
Listo n° 197
Listo n° 198
Listo n° 199
Listo n° 200
Listo n° 201
Listo n° 202
Listo n° 203
Listo n° 204
Listo n° 205
Listo n° 206
Listo n° 207
Listo n° 208
Listo n° 209
Listo n° 210
Listo n° 211
Listo n° 212
Listo n° 213
Listo n° 214
Listo n° 215
Listo n° 216
Listo n° 217
Listo n° 218
Listo n° 219
Listo n° 220
Listo n° 221
Listo n° 222
Listo n° 223
Listo n° 224
Listo n° 225
Listo n° 226
Listo n° 227
Listo n° 228
Listo n° 229
Listo n° 230
Listo n° 231
Listo n° 232
Listo n° 233
Listo n° 234
Listo n° 235
Tiempo transcurrido: 1985.6000082492828


In [119]:
df = df.replace('\n', '', regex=True)
path = r"C:\Users\User\OneDrive - Universidad del Pacífico\1. Documentos\0. Bases de datos\14. Conflictos sociales"
df.to_csv(path + "\dataConflitos.csv")

In [8]:
path = r"C:\Users\User\OneDrive - Universidad del Pacífico\1. Documentos\0. Bases de datos\14. Conflictos sociales"

df = pd.read_csv(path + "\dataConflitos.csv")

In [120]:
df['fecha_correcta'] = df['Ingreso como caso nuevo'].str.contains(r'.*2.*0.*')
df['dialogo_correcto'] = df['Dialogo'].str.contains(r'.*DIALOGO.*')


In [121]:
df[df['fecha_correcta']==0]

,Dpto,Tipo,Ingreso como caso nuevo,Caso,Ubicacion,Actores primarios,Actores secundarios,Actores terciarios,Dialogo,Detalle del mes,Mes,NInforme,fecha_correcta,dialogo_correcto
31,Cusco,Ingreso como caso nuevoMarzo de 2019,DialogoHAYDIALOGO,"""El alcald e provincial, los alcaldes distrit...",TipoSocioambiental,"Alcalde provincial de Chumbivilcas, Alcalde di...",Organismo de Evaluacion y Fiscalizacion Ambien...,"Comision de Ministros del MTC, MIDIS y MINEM, ...","CasoEl alcalde provincial, los alcaldes distri...",En pronunciamiento del 11 de junio del Frente ...,Mayo 2021,208,False,False
23,Cusco,Socioambiental,DialogoHAYDIALOGO,1. Se deja constancia que el MINEM entrega el...,"Distrito de Espinar, Provinci a de Espinar, re...","Comunidades campesinas de Huano Huano, Cala Ca...","Ministerio del Ambiente (MINAM), Ministerio de...",Secretaria de Gestion Social y Dialogo de la P...,"CasoLas comunidades campesinas de Huano Huano,...",El 19 de noviembre en la comunidad de Tintaya ...,Octubre 2019,189,False,False
102,Junin,Socioambiental,DialogoNOHAYDIALOGO,Las pobladores del distrito de San Jose de Qu...,"Distrito de San Jose de Quero, Provincia de C...","Municipalidad distrital de San Jose de Quero,...",Organismo de Evaluacion y Fiscalizacion Ambie...,-,Caso1) Se aprobo por unanimidad la conformaci...,El 25 de octubre se desarrollo la Mesa Tecnic...,Septiembre 2019,188,False,False
115,Puno,Socioambiental,DialogoNOHAYDIALOGO,1. CUMPLIR lo estipulado en la Ley Gestion In...,"Distrito de Cabanillas, provincia de San Roma...",Alcalde de la Municipalidad Distrital de Caba...,"Ministerio del Interior (MININTER),Ministerio...",-,Caso Autoridades municipales y organizaciones ...,"El 13 de octubre, la Defensoria del Pueblo est...",Septiembre 2019,188,False,False


In [162]:
df = datosConflictos(208)

In [163]:
print(df[0]["Detalle del mes"][31])

En pronunciamiento del 11 de junio del Frente Unico de 
Defensa de los Intereses de la Provincia de Chumbivilcas 
(FUDICH), el Frente Unico de Defensa de los Intereses del 
Distrito de Santo Tomas - FUDIST, La Liga Agraria del Distrito 
de Colquemarca, comunidade s campesinas del corredor 
minero y otras organizaciones sociales de base, manifestaron 
que aceptaron dialogar con la empresa Minera Las Bambas el 
10 de junio, sin embargo esta no asistio. Asimismo, 
mencionan la agenda que debe ser solucionada:1. Desconocimie nto de la R.M. Ndeg 372 -2018 -MTC2. Compensacion e indemnizacion por los danos y 
perjuicios ambientales, asimismo el pago de los 
derechos de servidumbre minera3. La empresa minera Las Bambas debe dar 
oportunidad a empresas comunales para el servicio 
de trans porte de mineral o linea amarilla, esto con la 
finalidad de lograr un autentico desarrollo comunal4. Inclusion como area de influencia directa ambiental y 

 


In [ ]:
df[3]